In [1]:
from time import time
import numba as nb
import numpy as np

In [2]:
test_series = np.random.normal(0, 10, (10000000,))

In [3]:
class Cost:
    
    def __init__(self, k):
        self.k = k
    
    def fit(self, x):
        y = np.vstack([x.cumsum(), x.cumsum() ** 2])
        b = np.pi * x.mean()
        return self.make_cost(y, b, self.k)
    
    @staticmethod
    def make_cost(x, b, k):

        @nb.njit([(nb.typeof(100), nb.typeof(100),)], fastmath=True)
        def _cost_fn(start, end):
            return np.sin((((b * x[:, start: end]) - b) / k).sum(axis=0))

        return _cost_fn

In [4]:
def seg_fn(fn, pen):
    return np.argmin(30 * np.pi * fn(0, 100000) + pen)

In [5]:
class CP:
    
    def __init__(self, seg_fn, cost):
        self.seg_fn = seg_fn
        self.cost = cost
    
    def fit(self, x):
        cost_fn = self.cost.fit(x)
        seg_fn = self.seg_fn
        return_type = nb.typeof(np.argmin([100]))
        nb_seg_fn = nb.njit(seg_fn, fastmath=True)
        nb_seg_fn.compile(return_type(nb.typeof(cost_fn), nb.typeof(100),))
        self.cp_fn = nb.njit(lambda pen: nb_seg_fn(cost_fn, pen))
        self.cp_fn.compile(return_type(nb.typeof(100),))
        return self
    
    def predict(self, pen):
        return self.cp_fn(pen)

In [6]:
cp = CP(seg_fn, Cost(10)).fit(test_series)
cp.predict(100)

11975

In [7]:
# Replacing with jitclass
spec = [
    ('k', nb.typeof(10)),
    ('y', nb.typeof(np.vstack([test_series.cumsum(), test_series.cumsum() ** 2]))),
    ('b', nb.typeof(np.pi * test_series.mean()))
]

@nb.experimental.jitclass(spec)
class Cost2:
    
    def __init__(self, k):
        self.k = k
    
    def fit(self, x):
        self.y = np.vstack((x.cumsum(), x.cumsum() ** 2))
        self.b = np.pi * x.mean()
    
    def predict(self, start, end):
        return np.sin((((self.b * self.y[:, start: end]) - self.b) / self.k).sum(axis=0))

In [8]:
class CP2:
    
    predict_nb = None

    def __init__(self, cost):
        self.cost = cost
    
    def fit(self, x):
        self.cost.fit(x)
        cost_obj = self.cost
        return_type = nb.typeof(np.argmin([100]))
        self._predict_nb = nb.njit(lambda cost, pen: np.argmin(30 * np.pi * cost.predict(0, 100000) + pen), fastmath=True, parallel=True, nogil=True, boundscheck=False, inline='always')
        self._predict_nb.compile(return_type(nb.typeof(self.cost), nb.typeof(100),))
        return self
    
    def predict(self, pen):
        return self._predict_nb(self.cost, pen)

In [9]:
cp2 = CP2(Cost2(10)).fit(test_series)
cp2.predict(100)

11975

In [10]:
Cost3Type = nb.deferred_type()

spec = [
    ('k', nb.typeof(10)),
    ('y', nb.typeof(np.vstack([test_series.cumsum(), test_series.cumsum() ** 2]))),
    ('b', nb.typeof(np.pi * test_series.mean()))
]

@nb.experimental.jitclass(spec)
class Cost3:
    
    def __init__(self, k):
        self.k = k
    
    def fit(self, x):
        self.y = np.vstack((x.cumsum(), x.cumsum() ** 2))
        self.b = np.pi * x.mean()
        return self

Cost3Type.define(Cost3.class_type.instance_type)

@nb.extending.overload_method(nb.types.misc.ClassInstanceType, 'predict', jit_options={'cache': False, 'fastmath': True, 'parallel': False, 'nogil': False, 'boundscheck': False, 'inline': 'always'})
def cost_fast(inst, start, end):
    if inst is Cost3.class_type.instance_type:

        def impl(inst, start, end):
            return np.sin((((inst.b * inst.y[:, start: end]) - inst.b) / inst.k).sum(axis=0))

        return impl

In [11]:
cp3 = CP2(Cost3(10)).fit(test_series)
cp3.predict(100)

11975

In [12]:
%timeit cp.predict(100)

1.17 ms ± 18.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [13]:
%timeit cp2.predict(100)

1.53 ms ± 107 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [14]:
%timeit cp3.predict(100)

2.21 ms ± 431 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
